# Import

In [15]:
! pip install faiss-cpu==1.7.4 mistralai

In [16]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API Key")
client = MistralClient(api_key=api_key)

Type your API Key··········


# Get data

In [17]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

# Chunkisation

In [18]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [19]:
len(chunks)

37

In [20]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
          model="mistral-embed",
          input=input
      )
    return embeddings_batch_response.data[0].embedding

In [21]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [22]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [23]:
text_embeddings

array([[-3.95202637e-02,  7.75756836e-02, -8.82148743e-05, ...,
        -1.26342773e-02, -2.12402344e-02, -2.50816345e-03],
       [-3.19213867e-02,  7.21435547e-02,  2.99835205e-02, ...,
        -1.08413696e-02, -1.19628906e-02, -7.66372681e-03],
       [-5.89599609e-02,  6.12487793e-02,  1.26419067e-02, ...,
        -2.25372314e-02,  4.67681885e-03, -6.26754761e-03],
       ...,
       [-5.52978516e-02,  6.89697266e-02,  2.69622803e-02, ...,
        -2.54211426e-02, -2.52227783e-02, -2.68859863e-02],
       [-3.90014648e-02,  5.63049316e-02,  4.76684570e-02, ...,
        -1.77001953e-02,  9.33074951e-03, -8.72039795e-03],
       [-2.99377441e-02,  5.81054688e-02,  1.70898438e-02, ...,
        -1.61132812e-02, -1.79290771e-02, -4.35791016e-02]])

In [24]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

# Question

In [25]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])

In [26]:
D, I = index.search(question_embeddings, k=2)

In [27]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

# Prompt

In [28]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [29]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [30]:
run_mistral(prompt)

'The two main things the author worked on before college were writing and programming. In terms of writing, the author wrote short stories, although they described them as being awful with little plot and mainly focusing on characters with strong feelings. As for programming, the author started writing programs in an early version of Fortran on an IBM 1401 in 9th grade, which they had access to in their school district\'s basement. However, the author notes that they couldn\'t figure out what to do with it and didn\'t have any data stored on punched cards, which limited their options. The author also mentions that during their first year of grad school, they realized that AI as practiced at the time was a hoax, and they decided to focus on Lisp instead, eventually writing a book about Lisp hacking called "On Lisp".'